In [4]:
import psycopg2 as p
import getpass, time
from datetime import datetime

In [5]:
start_time = time.time()

In [13]:
password = '*****'

conn = p.connect(database = 'postgres', user = 'postgres', password = password, 
                       host = 'localhost', port = 5433)

cur = conn.cursor()

## Updating the physical location information

In [14]:
# This query updates the location table and the fleet_primary table's location information
# Uses a CTE
# 1/24/2020
# 1/29/2020 changed it to only fleets between 1 and 19

cur.execute("""
with updated as (

    update fleetseek.location

    set 
        address1 = coalesce(sub_q.phy_address1, sub_q.mcmis_phy_address1),
        address2 = coalesce(sub_q.phy_suite, sub_q.phy_address2, sub_q.mcmis_phy_address2),
        address3 = null,
        city = coalesce(sub_q.phy_city, sub_q.mcmis_phy_city),
        county_fips = sub_q.phy_countyfips,
        state_code = coalesce(sub_q.phy_state, sub_q.mcmis_phy_state),
        zip_code = coalesce(sub_q.phy_zip_code, sub_q.mcmis_phy_zip),
        updated_date_time = now(),
        latitude = sub_q.phy_latitude,
        longitude = sub_q.phy_longitude,
        county_name = sub_q.phy_countyname

    from
    (
        select 
        

            address_difference,
            uniform_mcmis_address,
            fs_fleet_id,
            f.company_name,
            m.phy_str, 
            fp.primary_physical_location_id,
            fp.primary_mailing_location_id,
            f.total_vehicles,

            regexp_replace(
                regexp_replace(
                    regexp_replace(
                        regexp_replace(
                            regexp_replace(
                                regexp_replace(
                                    regexp_replace(
                                        regexp_replace(
                                            regexp_replace(
                                                regexp_replace(
                                                    regexp_replace(
                                                        regexp_replace(
                                                            regexp_replace(
                                                                regexp_replace(
                                                                    regexp_replace(
                                                                        regexp_replace(
                                                                            regexp_replace(
                                                                                regexp_replace(
                                                                                    regexp_replace(
                                                                                        initcap(ubci.phy_address), 'Avenue', 'Ave'), 
                                                                                    ' Suite ', ' Ste '), 
                                                                                E'#.*', ''), 
                                                                            E' Ste.*', ''), 
                                                                        E' Apt.*', ''), 
                                                                    'Street', 'St'), 
                                                                'Road', 'Rd'), 
                                                            'Drive', 'Dr'), 
                                                        'Lane', 'Ln'),
                                                    'Circle', 'Cir'),
                                                E'Unit.*', ''),
                                            ' Se ', ' SE '),
                                        ' Sw ', ' SW '),
                                    ' Nw ', ' NW '),
                                ' Ne ', ' NE '),
                            ' Ne$', ' NE '),
                        ' Se$', ' SE '),
                    ' Nw$', ' NW '),
                ' Sw$', ' SW ') as phy_address1, 
            l.address1 as location_address,
            case 
                when (ubci.phy_address ilike '% ste %' or phy_address ilike '%suite%' or phy_address ~ 'ste\s\d+ ' or phy_address ~ 'ste\#\d*' or phy_address ~ 'ste\s\w*' or phy_address ~ ' ste\d*')then concat('Ste ', regexp_replace(phy_address, '^.* ', '')) 
            end as phy_address2,
            phy_suite, 
            ubci.phy_city, 
            ubci.phy_countyfips, 
            ubci.phy_countyname, 
            ubci.phy_latitude, 
            ubci.phy_longitude, 
            ubci.phy_state,
            ubci.phy_zip_code,
            fp.fleet_id,
            l.location_id as location_id1,
            regexp_replace(
                regexp_replace(
                    regexp_replace(
                        regexp_replace(
                            regexp_replace(
                                regexp_replace(
                                    regexp_replace(
                                        regexp_replace(
                                            regexp_replace(
                                                regexp_replace(
                                                    regexp_replace(
                                                        regexp_replace(
                                                            regexp_replace(
                                                                regexp_replace(
                                                                    regexp_replace(
                                                                        regexp_replace(
                                                                            regexp_replace(
                                                                                regexp_replace(
                                                                                    regexp_replace(
                                                                                        initcap(m.phy_str), 'Avenue', 'Ave'), 
                                                                                    ' Suite ', ' Ste '), 
                                                                                E'#.*', ''), 
                                                                            E' Ste.*', ''), 
                                                                        E' Apt.*', ''), 
                                                                    'Street', 'St'), 
                                                                'Road', 'Rd'), 
                                                            'Drive', 'Dr'), 
                                                        'Lane', 'Ln'),
                                                    'Circle', 'Cir'),
                                                E'Unit.*', ''),
                                            ' Se ', ' SE '),
                                        ' Sw ', ' SW '),
                                    ' Nw ', ' NW '),
                                ' Ne ', ' NE '),
                            ' Ne$', ' NE '),
                        ' Se$', ' SE '),
                    ' Nw$', ' NW '),
                ' Sw$', ' SW ') as mcmis_phy_address1, 
            case 
                when (m.phy_str ilike '% ste %' or phy_str ilike '%suite%' or phy_str ~ 'ste\s\d+ ' or phy_str ~ 'ste\#\d*' or phy_str ~ 'ste\s\w*' or phy_str ~ ' ste\d*')then concat('Ste ', regexp_replace(phy_str, '^.* ', '')) 
            end as mcmis_phy_address2,
            m.phy_city as mcmis_phy_city,
            m.phy_st as mcmis_phy_state,
            m.phy_zip as mcmis_phy_zip

        from fleetseek.updates_bn_coder_input as ubci
        join fleetseek.fleet_primary as fp
        on ubci.fs_fleet_id = fp.fleet_id
        join fleetseek.location as l
        on fp.primary_physical_location_id = l.location_id
        join fleetseek.fleet as f
        on fp.fleet_id = f.fleet_id
        join mcmis as m
        on f.usdot = m.census_num
        where 
            ubci.address_difference = 'physical' or
            ubci.address_difference = 'both'

    ) as sub_q
    where 
        location.location_id = sub_q.location_id1 and
        sub_q.total_vehicles between 1 and 19 and
        sub_q.mcmis_phy_state != 'TA'
    returning
        city,
        state_code,
        county_fips, 
        zip_code,
        latitude,
        longitude,
        county_name,
        location_id1



)
update fleetseek.fleet_primary
set 
    city = initcap(updated.city),
    state_code = updated.state_code,
    county_fips = updated.county_fips,
    zip_code = updated.zip_code,
    latitude = updated.latitude,
    longitude = updated.longitude,
    city_display = case when(updated.county_name is null) then concat(initcap(updated.city), ', ', updated.state_code)
                        else concat(initcap(updated.city), ', ', updated.state_code, ' (', updated.county_name, ') ') end,
    update_date = now()::date

from updated
where 
    fleet_primary.primary_physical_location_id = updated.location_id1 and
    updated.state_code != 'TA'


""")
conn.commit()

In [15]:
# 1/29/2020: for fleets between 20 and 49 that have the same state code but a different address
# Uses CTE to update fleet_primary table
cur.execute("""
with updated as (

    update fleetseek.location

    set 
        address1 = coalesce(sub_q.phy_address1, sub_q.mcmis_phy_address1),
        address2 = coalesce(sub_q.phy_suite, sub_q.phy_address2, sub_q.mcmis_phy_address2),
        address3 = null,
        city = coalesce(sub_q.phy_city, sub_q.mcmis_phy_city),
        county_fips = sub_q.phy_countyfips,
        state_code = coalesce(sub_q.phy_state, sub_q.mcmis_phy_state),
        zip_code = coalesce(sub_q.phy_zip_code, sub_q.mcmis_phy_zip),
        updated_date_time = now(),
        latitude = sub_q.phy_latitude,
        longitude = sub_q.phy_longitude,
        county_name = sub_q.phy_countyname

    from
    (
        select 
            
            f.total_vehicles,
            l.state_code,
            m.phy_st,
            address_difference,
            uniform_mcmis_address,
            fs_fleet_id,
            f.company_name,
            m.phy_str, 
            fp.primary_physical_location_id,
            fp.primary_mailing_location_id,

            regexp_replace(
                regexp_replace(
                    regexp_replace(
                        regexp_replace(
                            regexp_replace(
                                regexp_replace(
                                    regexp_replace(
                                        regexp_replace(
                                            regexp_replace(
                                                regexp_replace(
                                                    regexp_replace(
                                                        regexp_replace(
                                                            regexp_replace(
                                                                regexp_replace(
                                                                    regexp_replace(
                                                                        regexp_replace(
                                                                            regexp_replace(
                                                                                regexp_replace(
                                                                                    regexp_replace(
                                                                                        initcap(ubci.phy_address), 'Avenue', 'Ave'), 
                                                                                    ' Suite ', ' Ste '), 
                                                                                E'#.*', ''), 
                                                                            E' Ste.*', ''), 
                                                                        E' Apt.*', ''), 
                                                                    'Street', 'St'), 
                                                                'Road', 'Rd'), 
                                                            'Drive', 'Dr'), 
                                                        'Lane', 'Ln'),
                                                    'Circle', 'Cir'),
                                                E'Unit.*', ''),
                                            ' Se ', ' SE '),
                                        ' Sw ', ' SW '),
                                    ' Nw ', ' NW '),
                                ' Ne ', ' NE '),
                            ' Ne$', ' NE '),
                        ' Se$', ' SE '),
                    ' Nw$', ' NW '),
                ' Sw$', ' SW ') as phy_address1, 
            l.address1 as location_address,
            case 
                when (ubci.phy_address ilike '% ste %' or phy_address ilike '%suite%' or phy_address ~ 'ste\s\d+ ' or phy_address ~ 'ste\#\d*' or phy_address ~ 'ste\s\w*' or phy_address ~ ' ste\d*')then concat('Ste ', regexp_replace(phy_address, '^.* ', '')) 
            end as phy_address2,
            phy_suite, 
            ubci.phy_city, 
            ubci.phy_countyfips, 
            ubci.phy_countyname, 
            ubci.phy_latitude, 
            ubci.phy_longitude, 
            ubci.phy_state,
            ubci.phy_zip_code,
            fp.fleet_id,
            l.location_id as location_id1,
            regexp_replace(
                regexp_replace(
                    regexp_replace(
                        regexp_replace(
                            regexp_replace(
                                regexp_replace(
                                    regexp_replace(
                                        regexp_replace(
                                            regexp_replace(
                                                regexp_replace(
                                                    regexp_replace(
                                                        regexp_replace(
                                                            regexp_replace(
                                                                regexp_replace(
                                                                    regexp_replace(
                                                                        regexp_replace(
                                                                            regexp_replace(
                                                                                regexp_replace(
                                                                                    regexp_replace(
                                                                                        initcap(m.phy_str), 'Avenue', 'Ave'), 
                                                                                    ' Suite ', ' Ste '), 
                                                                                E'#.*', ''), 
                                                                            E' Ste.*', ''), 
                                                                        E' Apt.*', ''), 
                                                                    'Street', 'St'), 
                                                                'Road', 'Rd'), 
                                                            'Drive', 'Dr'), 
                                                        'Lane', 'Ln'),
                                                    'Circle', 'Cir'),
                                                E'Unit.*', ''),
                                            ' Se ', ' SE '),
                                        ' Sw ', ' SW '),
                                    ' Nw ', ' NW '),
                                ' Ne ', ' NE '),
                            ' Ne$', ' NE '),
                        ' Se$', ' SE '),
                    ' Nw$', ' NW '),
                ' Sw$', ' SW ') as mcmis_phy_address1, 
            case 
                when (m.phy_str ilike '% ste %' or phy_str ilike '%suite%' or phy_str ~ 'ste\s\d+ ' or phy_str ~ 'ste\#\d*' or phy_str ~ 'ste\s\w*' or phy_str ~ ' ste\d*')then concat('Ste ', regexp_replace(phy_str, '^.* ', '')) 
            end as mcmis_phy_address2,
            m.phy_city as mcmis_phy_city,
            m.phy_st as mcmis_phy_state,
            m.phy_zip as mcmis_phy_zip

        from fleetseek.updates_bn_coder_input as ubci
        join fleetseek.fleet_primary as fp
        on ubci.fs_fleet_id = fp.fleet_id
        join fleetseek.location as l
        on fp.primary_physical_location_id = l.location_id
        join fleetseek.fleet as f
        on fp.fleet_id = f.fleet_id
        join mcmis as m
        on f.usdot = m.census_num
        where 
            ubci.address_difference = 'physical' or
            ubci.address_difference = 'both'

    ) as sub_q
    where 
        location.location_id = sub_q.location_id1 and
        sub_q.total_vehicles between 20 and 49 and
        sub_q.state_code = sub_q.phy_st
    
    returning
        city,
        location.state_code,
        county_fips, 
        zip_code,
        latitude,
        longitude,
        county_name,
        location_id1

)
update fleetseek.fleet_primary
set 
    city = initcap(updated.city),
    state_code = updated.state_code,
    county_fips = updated.county_fips,
    zip_code = updated.zip_code,
    latitude = updated.latitude,
    longitude = updated.longitude,
    city_display = case when(updated.county_name is null) then concat(initcap(updated.city), ', ', updated.state_code)
                        else concat(initcap(updated.city), ', ', updated.state_code, ' (', updated.county_name, ') ') end,
    update_date = now()::date

from updated
where fleet_primary.primary_physical_location_id = updated.location_id1



""")
conn.commit()

## Mailing location information

#### No CTE for these next two tables because the fleet_primary location information comes from the physical location information

In [16]:
# Updating mailing location information
# 1/24/2020
# 1/29/2020: changed to only fleets between 1 and 19. All information automatically updated regardless

cur.execute("""
update fleetseek.location

set 
    address1 = coalesce(sub_q.mai_address1, sub_q.mcmis_mai_address1),
    address2 = coalesce(sub_q.mai_suite, sub_q.mai_address2, sub_q.mcmis_mai_address2),
    address3 = null,
    city = coalesce(sub_q.mai_city, sub_q.mcmis_mai_city),
    county_fips = sub_q.mai_countyfips,
    state_code = coalesce(sub_q.mai_state, sub_q.mcmis_mai_state),
    zip_code = coalesce(sub_q.mai_zip_code, sub_q.mcmis_mai_zip),
    updated_date_time = now(),
    latitude = sub_q.mai_latitude,
    longitude = sub_q.mai_longitude,
    county_name = sub_q.mai_countyname

from 
(
    select 

        address_difference,
        uniform_mcmis_address,
        fs_fleet_id,
        f.total_vehicles,

        regexp_replace(
            regexp_replace(
                regexp_replace(
                    regexp_replace(
                        regexp_replace(
                            regexp_replace(
                                regexp_replace(
                                    regexp_replace(
                                        regexp_replace(
                                            regexp_replace(
                                                regexp_replace(
                                                    regexp_replace(
                                                        regexp_replace(
                                                            regexp_replace(
                                                                regexp_replace(
                                                                    regexp_replace(
                                                                        regexp_replace(
                                                                            regexp_replace(
                                                                                regexp_replace(
                                                                                    initcap(ubci.mai_address), 'Avenue', 'Ave'), 
                                                                                ' Suite ', ' Ste '), 
                                                                            E'#.*', ''), 
                                                                        E' Ste.*', ''), 
                                                                    E' Apt.*', ''), 
                                                                'Street', 'St'), 
                                                            'Road', 'Rd'), 
                                                        'Drive', 'Dr'), 
                                                    'Lane', 'Ln'),
                                                'Circle', 'Cir'),
                                            E'Unit.*', ''),
                                        ' Se ', ' SE '),
                                    ' Sw ', ' SW '),
                                ' Nw ', ' NW '),
                            ' Ne ', ' NE '),
                        ' Ne$', ' NE '),
                    ' Se$', ' SE '),
                ' Nw$', ' NW '),
            ' Sw$', ' SW ') as mai_address1, 
        l.address1 as location_address,
        case 
            when (ubci.mai_address ilike '% ste %' or mai_address ilike '%suite%' or mai_address ~ 'ste\s\d+ ' or mai_address ~ 'ste\#\d*' or mai_address ~ 'ste\s\w*' or mai_address ~ ' ste\d*')then concat('Ste ', regexp_replace(mai_address, '^.* ', '')) 
        end as mai_address2,
        mai_suite, 
        ubci.mai_city, 
        ubci.mai_countyfips, 
        ubci.mai_countyname, 
        ubci.mai_latitude, 
        ubci.mai_longitude, 
        ubci.mai_state,
        ubci.mai_zip_code,
        fp.fleet_id,
        l.location_id,
        regexp_replace(
            regexp_replace(
                regexp_replace(
                    regexp_replace(
                        regexp_replace(
                            regexp_replace(
                                regexp_replace(
                                    regexp_replace(
                                        regexp_replace(
                                            regexp_replace(
                                                regexp_replace(
                                                    regexp_replace(
                                                        regexp_replace(
                                                            regexp_replace(
                                                                regexp_replace(
                                                                    regexp_replace(
                                                                        regexp_replace(
                                                                            regexp_replace(
                                                                                regexp_replace(
                                                                                    initcap(m.mai_str), 'Avenue', 'Ave'), 
                                                                                ' Suite ', ' Ste '), 
                                                                            E'#.*', ''), 
                                                                        E' Ste.*', ''), 
                                                                    E' Apt.*', ''), 
                                                                'Street', 'St'), 
                                                            'Road', 'Rd'), 
                                                        'Drive', 'Dr'), 
                                                    'Lane', 'Ln'),
                                                'Circle', 'Cir'),
                                            E'Unit.*', ''),
                                        ' Se ', ' SE '),
                                    ' Sw ', ' SW '),
                                ' Nw ', ' NW '),
                            ' Ne ', ' NE '),
                        ' Ne$', ' NE '),
                    ' Se$', ' SE '),
                ' Nw$', ' NW '),
            ' Sw$', ' SW ') as mcmis_mai_address1, 
        case 
            when (m.mai_str ilike '% ste %' or mai_str ilike '%suite%' or mai_str ~ 'ste\s\d+ ' or mai_str ~ 'ste\#\d*' or mai_str ~ 'ste\s\w*' or mai_str ~ ' ste\d*')then concat('Ste ', regexp_replace(mai_str, '^.* ', '')) 
        end as mcmis_mai_address2,
        m.mai_city as mcmis_mai_city,
        m.mai_st as mcmis_mai_state,
        m.mai_zip as mcmis_mai_zip

    from fleetseek.updates_bn_coder_input as ubci
    join fleetseek.fleet_primary as fp
    on ubci.fs_fleet_id = fp.fleet_id
    join fleetseek.location as l
    on fp.primary_mailing_location_id = l.location_id
    join fleetseek.fleet as f
    on fp.fleet_id = f.fleet_id
    join mcmis as m
    on f.usdot = m.census_num
    where 
        (ubci.address_difference = 'mailing' or
        ubci.address_difference = 'both')
        
) as sub_q
where 
    fleetseek.location.location_id = sub_q.location_id and
    sub_q.total_vehicles between 1 and 19

""")
conn.commit()

In [17]:
# Fleets between 20 and 49 that have the same state code
cur.execute("""


update fleetseek.location

set 
    address1 = coalesce(sub_q.mai_address1, sub_q.mcmis_mai_address1),
    address2 = coalesce(sub_q.mai_suite, sub_q.mai_address2, sub_q.mcmis_mai_address2),
    address3 = null,
    city = coalesce(sub_q.mai_city, sub_q.mcmis_mai_city),
    county_fips = sub_q.mai_countyfips,
    state_code = coalesce(sub_q.mai_state, sub_q.mcmis_mai_state),
    zip_code = coalesce(sub_q.mai_zip_code, sub_q.mcmis_mai_zip),
    updated_date_time = now(),
    latitude = sub_q.mai_latitude,
    longitude = sub_q.mai_longitude,
    county_name = sub_q.mai_countyname

from 
(
    select 

        address_difference,
        uniform_mcmis_address,
        fs_fleet_id,
        f.total_vehicles,
        l.state_code,
        m.mai_st,

        regexp_replace(
            regexp_replace(
                regexp_replace(
                    regexp_replace(
                        regexp_replace(
                            regexp_replace(
                                regexp_replace(
                                    regexp_replace(
                                        regexp_replace(
                                            regexp_replace(
                                                regexp_replace(
                                                    regexp_replace(
                                                        regexp_replace(
                                                            regexp_replace(
                                                                regexp_replace(
                                                                    regexp_replace(
                                                                        regexp_replace(
                                                                            regexp_replace(
                                                                                regexp_replace(
                                                                                    initcap(ubci.mai_address), 'Avenue', 'Ave'), 
                                                                                ' Suite ', ' Ste '), 
                                                                            E'#.*', ''), 
                                                                        E' Ste.*', ''), 
                                                                    E' Apt.*', ''), 
                                                                'Street', 'St'), 
                                                            'Road', 'Rd'), 
                                                        'Drive', 'Dr'), 
                                                    'Lane', 'Ln'),
                                                'Circle', 'Cir'),
                                            E'Unit.*', ''),
                                        ' Se ', ' SE '),
                                    ' Sw ', ' SW '),
                                ' Nw ', ' NW '),
                            ' Ne ', ' NE '),
                        ' Ne$', ' NE '),
                    ' Se$', ' SE '),
                ' Nw$', ' NW '),
            ' Sw$', ' SW ') as mai_address1, 
        l.address1 as location_address,
        case 
            when (ubci.mai_address ilike '% ste %' or mai_address ilike '%suite%' or mai_address ~ 'ste\s\d+ ' or mai_address ~ 'ste\#\d*' or mai_address ~ 'ste\s\w*' or mai_address ~ ' ste\d*')then concat('Ste ', regexp_replace(mai_address, '^.* ', '')) 
        end as mai_address2,
        mai_suite, 
        ubci.mai_city, 
        ubci.mai_countyfips, 
        ubci.mai_countyname, 
        ubci.mai_latitude, 
        ubci.mai_longitude, 
        ubci.mai_state,
        ubci.mai_zip_code,
        fp.fleet_id,
        l.location_id,
        regexp_replace(
            regexp_replace(
                regexp_replace(
                    regexp_replace(
                        regexp_replace(
                            regexp_replace(
                                regexp_replace(
                                    regexp_replace(
                                        regexp_replace(
                                            regexp_replace(
                                                regexp_replace(
                                                    regexp_replace(
                                                        regexp_replace(
                                                            regexp_replace(
                                                                regexp_replace(
                                                                    regexp_replace(
                                                                        regexp_replace(
                                                                            regexp_replace(
                                                                                regexp_replace(
                                                                                    initcap(m.mai_str), 'Avenue', 'Ave'), 
                                                                                ' Suite ', ' Ste '), 
                                                                            E'#.*', ''), 
                                                                        E' Ste.*', ''), 
                                                                    E' Apt.*', ''), 
                                                                'Street', 'St'), 
                                                            'Road', 'Rd'), 
                                                        'Drive', 'Dr'), 
                                                    'Lane', 'Ln'),
                                                'Circle', 'Cir'),
                                            E'Unit.*', ''),
                                        ' Se ', ' SE '),
                                    ' Sw ', ' SW '),
                                ' Nw ', ' NW '),
                            ' Ne ', ' NE '),
                        ' Ne$', ' NE '),
                    ' Se$', ' SE '),
                ' Nw$', ' NW '),
            ' Sw$', ' SW ') as mcmis_mai_address1, 
        case 
            when (m.mai_str ilike '% ste %' or mai_str ilike '%suite%' or mai_str ~ 'ste\s\d+ ' or mai_str ~ 'ste\#\d*' or mai_str ~ 'ste\s\w*' or mai_str ~ ' ste\d*')then concat('Ste ', regexp_replace(mai_str, '^.* ', '')) 
        end as mcmis_mai_address2,
        m.mai_city as mcmis_mai_city,
        m.mai_st as mcmis_mai_state,
        m.mai_zip as mcmis_mai_zip

    from fleetseek.updates_bn_coder_input as ubci
    join fleetseek.fleet_primary as fp
    on ubci.fs_fleet_id = fp.fleet_id
    join fleetseek.location as l
    on fp.primary_mailing_location_id = l.location_id
    join fleetseek.fleet as f
    on fp.fleet_id = f.fleet_id
    join mcmis as m
    on f.usdot = m.census_num
    where 
        (ubci.address_difference = 'mailing' or
        ubci.address_difference = 'both')

) as sub_q
where 
    fleetseek.location.location_id = sub_q.location_id and
    sub_q.total_vehicles between 20 and 49 and
    sub_q.state_code = sub_q.mai_st

""")
conn.commit()